# Data Collection and Analysis
## Roman Urdu to Urdu Script Conversion Project

This notebook covers Step 1 & 2 of our methodology:
- Data Collection
- Data Analysis and Preprocessing

### Objectives:
1. Load and explore the Roman Urdu-Urdu parallel data
2. Analyze the dictionary mappings
3. Understand data characteristics and quality
4. Perform preprocessing and normalization
5. Generate statistics and visualizations

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path('../')
sys.path.append(str(project_root))

from utils.data_loader import DataLoader
from utils.preprocessing import RomanUrduPreprocessor
from utils.urdu_utils import UrduTextProcessor

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Data Loading and Initial Exploration

In [ ]:
# Initialize data loader and processors
data_loader = DataLoader("../data")
preprocessor = RomanUrduPreprocessor()
urdu_processor = UrduTextProcessor()

# Load all data
dictionary = data_loader.load_dictionary()
sample_data = data_loader.load_sample_data()
test_data = data_loader.load_test_data()

print(f"Dictionary size: {len(dictionary)}")
print(f"Sample data size: {len(sample_data)}")
print(f"Test data size: {len(test_data)}")

### Dictionary Analysis

In [ ]:
# Display sample dictionary entries
print("Sample Dictionary Entries:")
print("-" * 40)
sample_entries = list(dictionary.items())[:20]
for roman, urdu in sample_entries:
    print(f"{roman:15} -> {urdu}")

print(f"\nTotal dictionary entries: {len(dictionary)}")

In [ ]:
# Analyze word lengths in dictionary
roman_lengths = [len(word) for word in dictionary.keys()]
urdu_lengths = [len(word) for word in dictionary.values()]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Roman word lengths
ax1.hist(roman_lengths, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax1.set_title('Distribution of Roman Word Lengths')
ax1.set_xlabel('Word Length (characters)')
ax1.set_ylabel('Frequency')
ax1.grid(True, alpha=0.3)

# Urdu word lengths
ax2.hist(urdu_lengths, bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
ax2.set_title('Distribution of Urdu Word Lengths')
ax2.set_xlabel('Word Length (characters)')
ax2.set_ylabel('Frequency')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Roman words - Mean length: {np.mean(roman_lengths):.2f}, Max: {max(roman_lengths)}, Min: {min(roman_lengths)}")
print(f"Urdu words - Mean length: {np.mean(urdu_lengths):.2f}, Max: {max(urdu_lengths)}, Min: {min(urdu_lengths)}")

### Sample Data Analysis

In [ ]:
# Convert to DataFrame for easier analysis
df_sample = pd.DataFrame(sample_data)
df_test = pd.DataFrame(test_data)

print("Sample Data Structure:")
print(df_sample.head())

print("\nData Info:")
print(df_sample.info())

print("\nFirst 5 samples:")
for i, row in df_sample.head().iterrows():
    print(f"Roman: {row['roman']}")
    print(f"Urdu:  {row['urdu']}")
    print(f"English: {row['english']}")
    print("-" * 50)

In [ ]:
# Analyze sentence lengths
df_sample['roman_word_count'] = df_sample['roman'].apply(lambda x: len(x.split()))
df_sample['urdu_word_count'] = df_sample['urdu'].apply(lambda x: len(x.split()))
df_sample['roman_char_count'] = df_sample['roman'].apply(len)
df_sample['urdu_char_count'] = df_sample['urdu'].apply(len)

# Statistics
print("Sentence Length Statistics:")
print("=" * 30)
print(df_sample[['roman_word_count', 'urdu_word_count', 'roman_char_count', 'urdu_char_count']].describe())

In [ ]:
# Visualize sentence length distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Word counts
axes[0, 0].hist(df_sample['roman_word_count'], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Roman Sentence Word Count Distribution')
axes[0, 0].set_xlabel('Word Count')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(df_sample['urdu_word_count'], bins=15, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('Urdu Sentence Word Count Distribution')
axes[0, 1].set_xlabel('Word Count')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# Character counts
axes[1, 0].hist(df_sample['roman_char_count'], bins=15, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Roman Sentence Character Count Distribution')
axes[1, 0].set_xlabel('Character Count')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(df_sample['urdu_char_count'], bins=15, alpha=0.7, color='gold', edgecolor='black')
axes[1, 1].set_title('Urdu Sentence Character Count Distribution')
axes[1, 1].set_xlabel('Character Count')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Vocabulary Analysis

In [ ]:
# Extract vocabularies
roman_vocab = data_loader.get_vocabulary("sample", "roman")
urdu_vocab = data_loader.get_vocabulary("sample", "urdu")

print(f"Roman vocabulary size: {len(roman_vocab)}")
print(f"Urdu vocabulary size: {len(urdu_vocab)}")

# Most frequent words
all_roman_words = []
all_urdu_words = []

for item in sample_data:
    all_roman_words.extend(item['roman'].split())
    all_urdu_words.extend(item['urdu'].split())

roman_freq = Counter(all_roman_words)
urdu_freq = Counter(all_urdu_words)

print("\nTop 10 Roman words:")
for word, freq in roman_freq.most_common(10):
    print(f"{word:15}: {freq}")

print("\nTop 10 Urdu words:")
for word, freq in urdu_freq.most_common(10):
    print(f"{word:15}: {freq}")

In [ ]:
# Visualize word frequencies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Roman word frequencies
top_roman = dict(roman_freq.most_common(15))
ax1.barh(list(top_roman.keys()), list(top_roman.values()), color='skyblue')
ax1.set_title('Top 15 Roman Words', fontsize=14)
ax1.set_xlabel('Frequency')
ax1.grid(True, alpha=0.3)

# Urdu word frequencies
top_urdu = dict(urdu_freq.most_common(15))
ax2.barh(list(top_urdu.keys()), list(top_urdu.values()), color='lightcoral')
ax2.set_title('Top 15 Urdu Words', fontsize=14)
ax2.set_xlabel('Frequency')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Character Analysis

In [ ]:
# Character frequency analysis
roman_chars = data_loader.get_character_set("sample", "roman")
urdu_chars = data_loader.get_character_set("sample", "urdu")

print(f"Roman character set size: {len(roman_chars)}")
print(f"Urdu character set size: {len(urdu_chars)}")

print(f"\nRoman characters: {sorted(roman_chars)}")
print(f"\nUrdu characters: {sorted(urdu_chars)}")

In [ ]:
# Character frequency in all text
all_roman_text = ' '.join(item['roman'] for item in sample_data)
all_urdu_text = ' '.join(item['urdu'] for item in sample_data)

roman_char_freq = Counter(all_roman_text.lower())
urdu_char_freq = Counter(all_urdu_text)

# Remove spaces for cleaner visualization
roman_char_freq.pop(' ', None)
urdu_char_freq.pop(' ', None)

print("Top 10 Roman characters:")
for char, freq in roman_char_freq.most_common(10):
    print(f"'{char}': {freq}")

print("\nTop 10 Urdu characters:")
for char, freq in urdu_char_freq.most_common(10):
    print(f"'{char}': {freq}")

## 4. Data Quality Assessment

In [ ]:
# Check for data quality issues
quality_issues = []

for i, item in enumerate(sample_data):
    roman = item['roman']
    urdu = item['urdu']
    
    # Check for empty strings
    if not roman.strip() or not urdu.strip():
        quality_issues.append(f"Sample {i}: Empty text")
    
    # Check for very short sentences
    if len(roman.split()) < 2 or len(urdu.split()) < 2:
        quality_issues.append(f"Sample {i}: Very short sentence")
    
    # Check for very different lengths
    roman_words = len(roman.split())
    urdu_words = len(urdu.split())
    if abs(roman_words - urdu_words) > 2:
        quality_issues.append(f"Sample {i}: Length mismatch (R:{roman_words}, U:{urdu_words})")
    
    # Check for non-Urdu characters in Urdu text
    if not urdu_processor.is_urdu_text(urdu):
        quality_issues.append(f"Sample {i}: Non-Urdu characters detected")

print(f"Quality issues found: {len(quality_issues)}")
for issue in quality_issues[:10]:  # Show first 10
    print(f"  {issue}")

if len(quality_issues) > 10:
    print(f"  ... and {len(quality_issues) - 10} more")

## 5. Dictionary Coverage Analysis

In [ ]:
# Calculate dictionary coverage
covered_words = 0
total_words = 0
uncovered_words = set()

for item in sample_data:
    words = preprocessor.tokenize(item['roman'])
    total_words += len(words)
    
    for word in words:
        normalized_word = preprocessor.normalize_spelling(word.lower())
        if normalized_word in dictionary:
            covered_words += 1
        else:
            uncovered_words.add(word)

coverage_percentage = (covered_words / total_words) * 100 if total_words > 0 else 0

print(f"Dictionary Coverage Analysis:")
print(f"Total words in sample data: {total_words}")
print(f"Words covered by dictionary: {covered_words}")
print(f"Coverage percentage: {coverage_percentage:.2f}%")
print(f"Uncovered words: {len(uncovered_words)}")

print(f"\nSample uncovered words:")
for word in list(uncovered_words)[:20]:
    print(f"  {word}")

In [ ]:
# Visualize coverage
coverage_data = {
    'Covered': covered_words,
    'Uncovered': total_words - covered_words
}

plt.figure(figsize=(10, 6))
plt.pie(coverage_data.values(), labels=coverage_data.keys(), autopct='%1.1f%%', 
        colors=['lightgreen', 'lightcoral'], startangle=90)
plt.title('Dictionary Coverage of Sample Data', fontsize=16)
plt.axis('equal')
plt.show()

## 6. Preprocessing Analysis

In [ ]:
# Test preprocessing on sample sentences
test_sentences = [
    "aap kesy hain?",
    "main acha hun!",
    "ap kitab parh rahe hen",
    "wo ghar ja raha he"
]

print("Preprocessing Examples:")
print("=" * 50)

for sentence in test_sentences:
    cleaned = preprocessor.clean_text(sentence)
    normalized = preprocessor.normalize_spelling(sentence)
    tokenized = preprocessor.tokenize(sentence)
    
    print(f"Original:   {sentence}")
    print(f"Cleaned:    {cleaned}")
    print(f"Normalized: {normalized}")
    print(f"Tokenized:  {tokenized}")
    print("-" * 30)

In [ ]:
# Test spelling variations
test_words = ['kaise', 'kyun', 'main', 'aap', 'ghar']

print("Spelling Variations:")
print("=" * 30)

for word in test_words:
    variations = preprocessor.generate_variations(word)
    print(f"{word}: {variations}")

## 7. Data Statistics Summary

In [ ]:
# Generate comprehensive statistics
stats = data_loader.get_statistics("sample")

print("Comprehensive Data Statistics:")
print("=" * 40)
for key, value in stats.items():
    if isinstance(value, float):
        print(f"{key:30}: {value:.2f}")
    else:
        print(f"{key:30}: {value}")

# Create a summary report
summary_report = {
    'dataset_info': {
        'dictionary_size': len(dictionary),
        'sample_data_size': len(sample_data),
        'test_data_size': len(test_data),
        'coverage_percentage': coverage_percentage
    },
    'vocabulary_stats': {
        'roman_vocab_size': len(roman_vocab),
        'urdu_vocab_size': len(urdu_vocab),
        'roman_char_set_size': len(roman_chars),
        'urdu_char_set_size': len(urdu_chars)
    },
    'quality_assessment': {
        'quality_issues_count': len(quality_issues),
        'uncovered_words_count': len(uncovered_words)
    }
}

print("\n" + "=" * 50)
print("SUMMARY REPORT")
print("=" * 50)
for category, data in summary_report.items():
    print(f"\n{category.upper()}:")
    for key, value in data.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value}")

## 8. Export Processed Data

In [ ]:
# Save processed data and analysis results
import json

# Export summary report
with open('../data/data_analysis_report.json', 'w', encoding='utf-8') as f:
    json.dump(summary_report, f, ensure_ascii=False, indent=2)

# Export uncovered words for dictionary expansion
uncovered_words_list = {
    'uncovered_words': list(uncovered_words),
    'word_frequencies': dict(roman_freq.most_common(50))
}

with open('../data/uncovered_words.json', 'w', encoding='utf-8') as f:
    json.dump(uncovered_words_list, f, ensure_ascii=False, indent=2)

# Export processed sample data with additional features
df_sample.to_csv('../data/processed_sample_data.csv', index=False, encoding='utf-8')

print("Data analysis complete!")
print("Exported files:")
print("  - data_analysis_report.json")
print("  - uncovered_words.json")
print("  - processed_sample_data.csv")

## Conclusions

### Key Findings:
1. **Data Quality**: The dataset shows good overall quality with minimal issues
2. **Dictionary Coverage**: Coverage analysis reveals areas for improvement
3. **Vocabulary Characteristics**: Clear patterns in word and character distributions
4. **Preprocessing Effectiveness**: Normalization significantly improves consistency

### Next Steps:
1. Use uncovered words to expand the dictionary
2. Implement the dictionary-based conversion model
3. Train machine learning models on the processed data
4. Evaluate model performance using the test set

### Recommendations:
- Focus on high-frequency uncovered words for dictionary expansion
- Consider spelling variations in model development
- Use character-level information for handling unknown words